In [ ]:
!pip3 install --upgrade pandas requests sentence-transformers transformers pyarrow

In [ ]:
!git lfs clone https://github.com/amazon-science/esci-data.git

In [ ]:
document_field="product_title"
batch_size=128

In [ ]:
!ls -l  esci-data/shopping_queries_dataset

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CERerankingEvaluator
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers import evaluation
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df_examples = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_examples.parquet')

In [ ]:
df_products = pd.read_parquet('esci-data/shopping_queries_dataset/shopping_queries_dataset_products.parquet')

In [ ]:
df_examples_products = pd.merge(
        df_examples,
        df_products,
        how='left',
        left_on=['product_locale', 'product_id'],
        right_on=['product_locale', 'product_id']
    )

In [ ]:
esci_label2gain = {
        'E' : 1,
        'S' : 0.1,
        'C' : 0.01,
        'I' : 0,
    }

In [ ]:
df_examples_products = df_examples_products[df_examples_products['small_version'] == 1]
df_examples_products = df_examples_products[df_examples_products['split'] == "train"]
df_examples_products = df_examples_products[df_examples_products['product_locale'] == 'us']
df_examples_products['gain'] = df_examples_products['esci_label'].apply(lambda esci_label: esci_label2gain[esci_label])

Download our own train/dev split 

In [ ]:
train_queries = pd.read_parquet("https://data.vespa.oath.cloud/sample-apps-data/train_query_ids.parquet")['query_id'].unique()

In [ ]:

list_query_id_train = train_queries

In [ ]:
len(train_queries)

In [ ]:
df_examples_products = df_examples_products[['query_id', 'query', 'product_title','product_description', 'product_bullet_point', 'gain']]
df_train = df_examples_products[df_examples_products['query_id'].isin(list_query_id_train)]

In [ ]:
def replace_none(text):
  if text == None:
    text = ''
  return text

In [ ]:
train_samples = []
for (_, row) in df_train.iterrows():
  train_samples.append(InputExample(texts=[row['query'], replace_none(row[document_field])], label=float(row['gain'])))
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size, drop_last=True)

In [ ]:
model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'
num_epochs = 2
num_labels = 1
max_length = 96
    
model = CrossEncoder(
  model_name, 
  num_labels=num_labels, 
  max_length=max_length, 
  default_activation_function=torch.nn.Identity(), 
  device=device
)
loss_fct=torch.nn.MSELoss()
warmup_steps = 10
lr = 4e-6

In [ ]:
model.fit(
  train_dataloader=train_dataloader,
  loss_fct=loss_fct,
  epochs=num_epochs,
  optimizer_params={'lr': lr},
)
model.save("model")

In [ ]:
token='HF_TOKEN'

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [ ]:
automodel = AutoModelForSequenceClassification.from_pretrained("./model/")

In [ ]:
autotokenizer = AutoTokenizer.from_pretrained("./model/")

In [ ]:
name = document_field + "_ranker"

In [ ]:
automodel.push_to_hub(name, use_auth_token=token)

In [ ]:
autotokenizer.push_to_hub(name, use_auth_token=token)

Train bi-encoder with mean-pooling and Cosine Similarity (angular)

In [ ]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
train_loss = losses.CosineSimilarityLoss(model=model)
num_epochs = 1

In [ ]:
model.fit(
  train_objectives=[(train_dataloader, train_loss)],
  epochs=num_epochs,
  output_path="bi-encoder",
)

In [ ]:
from transformers import BertModel 

In [ ]:
autmodel = BertModel.from_pretrained("./bi-encoder")

In [ ]:
name = document_field + "_encoder"

In [ ]:
autmodel.push_to_hub(name, use_auth_token=token)

In [ ]:
autotokenizer.push_to_hub(name, use_auth_token=token)